In [1]:
%run plputil.py


modelseedpy 0.3.3
cobrakbase 0.3.1


In [2]:
%run plputil.py
ecoli_model = MSModelUtil.from_cobrapy_json("newdata/iML1515_PLP_Jun27_JTB.json")
util.msrecon.save_model(ecoli_model,186862,"iML1515_PLP")

Output files printed to:/Users/chenry/code/notebooks/ModelSEED2Notebooks/ADP1/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


1721156397.643206 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.


TypeError: unhashable type: 'list'

In [5]:
solution = modmodel.optimize()
print("Mod:",solution.objective_value)
solution = origmodel.optimize()
print("Original:",solution.objective_value)

Mod: 0.8769970445295686
Original: 0.87699721425716


In [2]:
condition = "pdxBJ_20nM/5µM"
#model= cobra.io.read_sbml_model("iML1515.xml")
model= cobra.io.load_json_model("iML1515_PLP_Jun27_JTB.json")
media_dict = {
    "glc__D":22.5,
    "nh4":100,
    "k":100,
    "pydxn":0.00002,
    'so4':100,
    'pi':100,
    'mn2': 100,
    'fe2': 100,
    'na1': 100,
    'mg2': 100,
    'o2': 20,
    'h2o': 100,
    'h': 100,
    'fe3': 100.0,
    'zn2': 100.0,
    'ca2': 100.0,
    'ni2': 100.0,
    'cu2': 100.0,
    'sel': 100.0,
    'cobalt2': 100.0,
    'mobd': 100.0,
    'cl': 100.0,
    'cbl1':100,
    'tungs': 100.0,
    'slnt': 100.0
}
media = MSMedia.from_dict(media_dict)
pkgmgr = MSPackageManager.get_pkg_mgr(model)
pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
#Plug in growth rate and minimize nutrient uptake
model.reactions.BIOMASS_Ec_iML1515_WT_75p37M.lower_bound = 0.3768
model.reactions.BIOMASS_Ec_iML1515_WT_75p37M.upper_bound = 0.3768
glucose_uptake_objective = model.problem.Objective(
    1 * model.reactions.EX_glc__D_e.flux_expression,
    direction='max')
model.objective = glucose_uptake_objective
solution = model.optimize()
#Constraining media to limit glucose to hit measured growth rate
media_dict["glc__D"] = -1*solution.fluxes["EX_glc__D_e"]
print(media_dict["glc__D"])
media = MSMedia.from_dict(media_dict)
pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
#Confirming that we now get the correct growth rate
model.reactions.BIOMASS_Ec_iML1515_WT_75p37M.lower_bound = 0
model.reactions.BIOMASS_Ec_iML1515_WT_75p37M.upper_bound = 100
biomass_objective = model.problem.Objective(
    1 * model.reactions.BIOMASS_Ec_iML1515_WT_75p37M.flux_expression,
    direction='max')
model.objective = biomass_objective
solution = cobra.flux_analysis.pfba(model)
print(solution.fluxes["BIOMASS_Ec_iML1515_WT_75p37M"])

4.447533484208366
0.3768000000000051


In [3]:
#Computing FVA and resetting bounds to be twice flux if flux to FVA max if not
fva = flux_variability_analysis(model,model.reactions,fraction_of_optimum=0.1)
for reaction in model.reactions:
    original = [reaction.lower_bound,reaction.upper_bound]
    if reaction.id[0:3] != "EX_" and reaction.id != "ATPM":
        if solution.fluxes[reaction.id] > 0:
            reaction.upper_bound = 2*solution.fluxes[reaction.id]
            reaction.lower_bound = 0
        elif solution.fluxes[reaction.id] < 0:
            reaction.lower_bound = 2*solution.fluxes[reaction.id]
            reaction.upper_bound = 0
        else:
            if fva["maximum"][reaction.id] <= 0:
                reaction.upper_bound = 0
            elif 2*fva["maximum"][reaction.id] < 1000:
                reaction.upper_bound = 2*fva["maximum"][reaction.id]
            if fva["minimum"][reaction.id] >= 0:
                reaction.lower_bound = 0
            elif 2*fva["minimum"][reaction.id] > -1000:
                reaction.lower_bound = 2*fva["minimum"][reaction.id]
    print(reaction.id,solution.fluxes[reaction.id],original[0],original[1],reaction.lower_bound,reaction.upper_bound)
solution = cobra.flux_analysis.pfba(model)
print(solution.fluxes["BIOMASS_Ec_iML1515_WT_75p37M"])

CYTDK2 0.0 0.0 1000.0 0 144.17843253481996
XPPT 0.0 0.0 1000.0 0 72.08921626741078
HXPRT 0.0 0.0 1000.0 0 72.08921626740998
NDPK5 0.0 -1000.0 1000.0 -50.79088559465709 72.09114638772992
SHK3Dr 0.14014096320000186 -1000.0 1000.0 0 0.2802819264000037
NDPK6 0.0 -1000.0 1000.0 0 32.597805132619094
NDPK8 0.0 -1000.0 1000.0 -50.79088559465709 72.09108557220976
DHORTS -0.12069808320000161 -1000.0 1000.0 -0.24139616640000322 0
OMPDC 0.12069808320000161 0.0 1000.0 0 0.24139616640000322
PYNP2r 0.0 -1000.0 1000.0 -144.17843253481988 72.09108557220999
G5SD 0.08093588640000109 0.0 1000.0 0 0.16187177280000217
CS 3.4129483997019054 0.0 1000.0 0 6.825896799403811
ICDHyr 3.4129483997019054 -1000.0 1000.0 0 6.825896799403811
ALATA_L2 0.0 0.0 1000.0 0 71.2942154802102
DURIPP 0.0 -1000.0 1000.0 -7.9602177517687 32.59593582781861
ACALD 0.0 -1000.0 1000.0 -144.17843253482005 320.82351484450294
PTRCTA 0.0 0.0 1000.0 0 27.090059000096687
ACS 0.0 0.0 1000.0 0 144.1784325348188
CYSDS 0.0 0.0 1000.0 0 47.555213

TRE6PH 0.0 0.0 1000.0 0 72.08921626740971
XYLI1 0.0 -1000.0 1000.0 0 0
PPC 1.1025277272018856 0.0 1000.0 0 2.2050554544037713
ICHORS_copy1 0.0 0.0 1000.0 0 1000.0
EX_sucr_e 0.0 0 100 0 100
ALDD4 0.0 0.0 1000.0 0 0
HACD4 0.11800697760000159 -1000.0 1000.0 0 0.23601395520000318
G3PT 0.0 0.0 1000.0 0 144.17843253482047
CINNDO 0.0 0.0 1000.0 0 0
DHPPD 0.0 0.0 1000.0 0 0
EX_tre_e 0.0 0 100 0 100
EX_zn2_e -0.00012208320000000166 -100.0 1000 -100.0 1000
HACD5 0.11800697760000159 -1000.0 1000.0 0 0.23601395520000318
DHCIND 0.0 0.0 1000.0 0 0
HACD7 0.09957919680000134 -1000.0 1000.0 0 0.19915839360000268
CITL 0.0 0.0 1000.0 0 144.17842302062033
GOFUCR 0.0 0.0 1000.0 0 0
PYDXK 0.0 0.0 1000.0 0 144.1784230206204
DNMPPA 0.0003361055999999815 0.0 1000.0 0 0.000672211199999963
HPPK2 0.0003361055999999815 0.0 1000.0 0 0.000672211199999963
PTPATi 0.00021703680000006997 0.0 1000.0 0 0.00043407360000013995
HEX7 0.0 0.0 1000.0 0 150.84972324693285
DXPRIi 0.002496676800000028 0.0 1000.0 0 0.00499335360000

U23GAAT 0.006142593600000083 0.0 1000.0 0 0.012285187200000165
EX_f6p_e 0.0 0 100 0 100
UAAGDS 0.010391013600000143 0.0 1000.0 0 0.020782027200000285
AACPS4 0.03555032640000048 0.0 1000.0 0 0.07110065280000095
UAMAS 0.010391013600000143 0.0 1000.0 0 0.020782027200000285
UAMAGS 0.010391013600000143 0.0 1000.0 0 0.020782027200000285
UGLYCH 0.0 0.0 1000.0 0 11.29572485832512
NTPP3 0.0 0.0 1000.0 0 72.08921626739918
NTPP6 0.0 0.0 1000.0 0 72.08921626739938
NTPP7 0.0 0.0 1000.0 0 72.08921626739921
OP4ENH 0.0 0.0 1000.0 0 0
EX_rmn_e 0.0 0 100 0 100
HOPNTAL 0.0 0.0 1000.0 0 0
NTPP8 0.0 0.0 1000.0 0 72.08921626740153
P5CR 0.08093588640000109 0.0 1000.0 0 0.16187177280000217
CYANST 0.0 0.0 1000.0 0 0
EX_ttdca_e 0.0 0 100 0 100
DHPPDA2 0.000168052799999938 0.0 1000.0 0 0.000336105599999876
3HPPPNH 0.0 0.0 1000.0 0 0
G5SADs 0.08093588640000109 0.0 1000.0 0 0.16187177280000217
GMAND 0.0 0.0 1000.0 0 0
PPCSCT 0.0 0.0 1000.0 0 1000.0
CYTD 0.0 0.0 1000.0 0 48.060100613210984
AKGDH 3.0029349869019 0.0

NO3R1bpp 0.0 0.0 1000.0 0 0
O2tpp 10.499558206449732 -1000.0 1000.0 0 20.999116412899465
EX_octa_e 0.0 0 100 0 100
PHYTSpp 0.0 0.0 1000.0 0 0
EX_gbbtn_e 0.0 0 100 0 100
PPTHpp 0.0 0.0 1000.0 0 0
PYRt2rpp 0.0 -1000.0 1000.0 -16.761931491713426 0
NO3R2pp 0.0 0.0 1000.0 0 0
EX_xylu__L_e 0.0 0 100 0 100
EX_gdp_e 0.0 0 100 0 100
EX_gtp_e 0.0 0 100 0 100
NTP3pp 0.0 0.0 1000.0 0 0
NO3R2bpp 0.0 0.0 1000.0 0 0
THMabcpp 0.0 0.0 1000.0 0 0
THMDt2pp_copy2 0.0 -1000.0 1000.0 -1000.0 0
THRabcpp 0.0 0.0 1000.0 0 115.34274602785496
TREptspp 0.0 0.0 1000.0 0 0
THD2pp 0.0 0.0 1000.0 0 288.3568650696371
TMAOR1pp 0.0 0.0 1000.0 0 0
TMAOR2pp 0.0 0.0 1000.0 0 0
XANtpp 0.0 -1000.0 1000.0 -576.7136920824746 0
DAPtex 0.0 -1000.0 1000.0 -5.978529283642878 0
TRPt2rpp 0.0 -1000.0 1000.0 -3.446114521043244 576.7137301392761
TYRt2rpp 0.0 -1000.0 1000.0 -4.364254948991406 576.7137301392751
EX_tma_e 0.0 0 100 0 100
URIt2pp_copy1 0.0 0.0 1000.0 0 1000.0
URIt2pp_copy2 0.0 -1000.0 1000.0 -1000.0 0
VALabcpp 0.0 0.0 1000.

L_LACD2 0.0 0.0 1000.0 0 39.06323343008794
ACACT6r 0.10265049360000138 -1000.0 1000.0 0 0.20530098720000275
TRPtex 0.0 -1000.0 1000.0 -3.4461145210432194 0
PTRCt2pp 0.0 0.0 1000.0 0 288.3568460412397
GGGABAH 0.0 0.0 1000.0 0 20.655911167314553
PNTOtex 0.0 -1000.0 1000.0 0 0
FRULYSK 0.0 0.0 1000.0 0 0
FDMO6 0.0 0.0 1000.0 0 0
TSULtex 0.0 -1000.0 1000.0 0 0
TYRtex 0.0 -1000.0 1000.0 -4.364254948991519 0
URAtex 0.0 -1000.0 1000.0 -12.095633749357342 0
UREAtex 0.0 -1000.0 1000.0 -18.632891267115106 0
URItex 0.0 -1000.0 1000.0 -5.024596137972236 0
SULFACabcpp 0.0 0.0 1000.0 0 0
ETHSO3tex 0.0 -1000.0 1000.0 0 0
BUTSO3tex 0.0 -1000.0 1000.0 0 0
LIPOtex 0.0 -1000.0 1000.0 0 0
CYANSTpp 0.0 0.0 1000.0 0 0
VALtex 0.0 -1000.0 1000.0 -7.719077990691793 0
XANtex 0.0 -1000.0 1000.0 -9.201212575077593 0
MCPST 0.0 0.0 1000.0 0 0
XYLtex 0.0 -1000.0 1000.0 0 0
PPPNtex 0.0 -1000.0 1000.0 0 0
ABUTtex 0.0 -1000.0 1000.0 -9.445996237204502 0
FORtppi 0.0 0.0 1000.0 0 576.7138433300013
PHEMEabcpp 0.0 0.0 1000.

...

LPLIPAL2G141 0.0 0.0 1000.0 0 14.417843253481879
2AGPA120tipp 0.0 0.0 1000.0 0 19.886680349630176
2AGPG180tipp 0.0 0.0 1000.0 0 28.83568650696376
IPDDI 0.0 -1000.0 1000.0 -0.004279694400000023 0.0007136592000000049
ORNDC 0.015077275200000205 0.0 1000.0 0 0.03015455040000041
UPP3MT 8.402640000000114e-05 0.0 1000.0 0 0.00016805280000000227
KAS15 0.0 0.0 1000.0 0 6.621732525342826
ALAALAtex 0.0 -1000.0 1000.0 -5.21126821689052 0
FACOAL181t2pp 0.0 0.0 1000.0 0 31.173713085539333
T2DECAI 0.0 -1000.0 1000.0 0 10.796868516224968
2AGPGAT180 0.0 0.0 1000.0 0 28.835686506963903
2AGPGAT181 0.0 0.0 1000.0 0 28.83568650696391
3OAS161 0.0 0.0 1000.0 0 17.37243970939147
DHQS 0.14014096320000186 0.0 1000.0 0 0.2802819264000037
EAR141y 0.0 0.0 1000.0 0 10.605774297188018
EAR161x 0.0 0.0 1000.0 0 17.372439709391365
2AGPG141tipp 0.0 0.0 1000.0 0 28.835686506963757
2AGPG161tipp 0.0 0.0 1000.0 0 28.835686506963775
2AGPG181tipp 0.0 0.0 1000.0 0 28.835686506963764
2AGPEAT140 0.0 0.0 1000.0 0 28.8356865069637

PGSA141 0.0 0.0 1000.0 0 28.835686506964176
EX_alaala_e 0.0 0 100 0 100
EX_dtmp_e 0.0 0 100 0 100
5DOAN 8.704080000000117e-05 0.0 1000.0 0 0.00017408160000000235
DM_5drib_c 8.704080000000117e-05 0.0 1000.0 0 0.00017408160000000235
EX_lipoate_e 0.0 0 100 0 100
EX_pydx_e 0.0 0 100 0 100
EX_g1p_e 0.0 0 100 0 100
GLCtex_copy2 0.0 -1000.0 1000.0 -1000.0 8.895066968416732
NAt3pp 0.0 0.0 1000.0 0 1000.0
MINOHPtexi 0.0 0.0 1000.0 0 0
12PPDRtpp 0.0 -1000.0 1000.0 -10.786096848425364 0
SFGTHi 0.0 0.0 1000.0 0 0
PYDAMK 0.0 0.0 1000.0 0 144.17843253481917
DHORDfum 0.0 0.0 1000.0 0 12.119775170145349
K2L4Aabcpp 0.0 0.0 1000.0 0 0.013158081675855225
FACOAE120 0.015356484000000207 0.0 1000.0 0 0.030712968000000413
3OAR120 0.0 -1000.0 1000.0 0 17.370051655839926
3HAD80 0.0 0.0 1000.0 0 6.621732525343043
3HAD120 0.0 0.0 1000.0 0 17.370051655840033
DSBCGT 0.0 0.0 1000.0 0 0
DSBGGT 0.0 0.0 1000.0 0 0
EAR180x 0.0 0.0 1000.0 0 17.368360217022595
AKGtex 0.0 -1000.0 1000.0 -10.05715889502813 0
SO4t2pp 0.0947

3OAS120 0.0 0.0 1000.0 0 17.370052964264655
LSERDHr 0.0 -1000.0 1000.0 0 66.71720710809768
3OAR40 0.0 -1000.0 1000.0 0 6.621733024134448
3OAR100 0.0 -1000.0 1000.0 0 11.866669846873648
DSERDHr 0.0 -1000.0 1000.0 -66.7172071080975 0
3HAD60 0.0 0.0 1000.0 0 6.621733024134517
3HAD100 0.0 0.0 1000.0 0 11.866669846873783
3HAD140 0.0 0.0 1000.0 0 17.369353325301145
3HAD160 0.0 0.0 1000.0 0 17.373263392007065
EX_urate_e 0.0 0 100 0 100
PSD120 0.0 0.0 1000.0 0 14.166567974508979
AGPAT180 0.0 0.0 1000.0 0 19.88668034963018
AGPAT141 0.0 0.0 1000.0 0 19.886680349630364
PAPA120 0.0 0.0 1000.0 0 144.17843253482
EX_skm_e 0.0 0 100 0 100
EX_cpgn_un_e 0.0 0 100 0 100
MDDEP1pp 0.0 0.0 1000.0 0 6.202050056914733
MDDEP2pp 0.0 0.0 1000.0 0 6.266652363454499
AGPAT161 0.01777516320000024 0.0 1000.0 0 0.03555032640000048
DASYN120 0.0 0.0 1000.0 0 72.0892162674094
EAR40x 0.0 0.0 1000.0 0 6.6217330241344206
EAR60y 0.0 0.0 1000.0 0 6.549364357204002
MLDEP1pp 0.0005067960000000068 0.0 1000.0 0 0.0010135920000000

MPTAT 5.275200000000071e-06 0.0 1000.0 0 1.0550400000000142e-05
APG3PAT141 0.0 0.0 1000.0 0 19.88667903732673
MOADSUx 1.0550400000000142e-05 0.0 1000.0 0 2.1100800000000284e-05
3HPPtex 0.0 -1000.0 1000.0 -9.064221817136074 0
CMtpp 0.0 0.0 1000.0 0 576.7137301392718
FUSAtpp 0.0 0.0 1000.0 0 576.7137301392718
APG3PAT181 0.0 0.0 1000.0 0 19.88851257731028
APH140 0.0 0.0 1000.0 0 72.08921626740972
APH160 0.0 0.0 1000.0 0 72.08921626740974
OPMEACPD 7.53600000000243e-07 0.0 1000.0 0 1.507200000000486e-06
EPMEACPR 7.536000000000101e-07 0.0 1000.0 0 1.5072000000000203e-06
RFAMPtex 0.0 -1000.0 1000.0 0 576.7137301392725
MINCYCtex 0.0 -1000.0 1000.0 0 576.7137301392725
AOXSr2 7.536000000000101e-07 0.0 1000.0 0 1.5072000000000203e-06
MINCYCtpp 0.0 0.0 1000.0 0 576.7137301392746
MEOHtrpp -7.536000000000101e-07 -1000.0 1000.0 -1.5072000000000203e-06 0
EX_LalaDglu_e 0.0 0 100 0 100
SCYSDS 0.0 0.0 1000.0 0 0.0008189978862019763
DOXRBCNtex 0.0 -1000.0 1000.0 0 576.7137301392751
DOXRBCNtpp 0.0 0.0 1000

SEPHCHCS 0.00016805280000000227 0.0 1000.0 0 0.00033610560000000455
DHNCOAS 0.00016805280000000227 0.0 1000.0 0 0.00033610560000000455
PCNO 0.0 -1000.0 1000.0 0 0
PYK6 0.012417820800000168 0.0 1000.0 0 0.024835641600000336
DHNCOAT 0.00016805280000000227 0.0 1000.0 0 0.00033610560000000455
FADt 0.0 0.0 1000.0 0 1.3650693856885707
HPACOAT 0.0 0.0 1000.0 0 0
PUACGAMtr 0.0 0.0 1000.0 0 0
DC6PDA 0.0 0.0 1000.0 0 0
5DGLCNR 0.0 0.0 1000.0 0 320.8235148445049
PIt2rpp 0.3548065608001201 0.0 1000.0 0 0.7096131216002401
FADtex 0.0 -1000.0 1000.0 0 1.3650693856885707
AI2abcpp 0.0 0.0 1000.0 0 144.17843253481865
PAI2I 0.0005049120000000067 0.0 1000.0 0 0.0010098240000000133
4HBZt3pp 0.0 0.0 1000.0 0 0
CURR 0.0 0.0 1000.0 0 0
LKDRA 0.0 -1000.0 1000.0 0 0
LEUt2rpp 0.0 0.0 1000.0 0 288.3568650696386
LCTStpp 0.0 0.0 1000.0 0 288.3568650696397
ILEt3pp 0.0 0.0 1000.0 0 288.35686506963947
METNA 0.0 -1000.0 1000.0 0 0
METt3pp 0.0 0.0 1000.0 0 115.34274602785567
FESD1s 0.0013994352000000188 0.0 1000.0 0 0.0

In [3]:
#Add the gene expression profile to the project notebook folder (Fatime)
expression_data = MSExpression.from_gene_feature_file("differentialexpression_gene.tsv",None,False)
#Covert differential gene expression profile into differential reaction expression profile (Chris)
reaction_expression = expression_data.build_reaction_expression(model,0)
#Need to load the metabolite differential profile data in terms of iML1515 IDs
metabolite_data = MSExpression.from_gene_feature_file("Metabolomics.tsv",None,False)
modelutl = MSModelUtil(model)
exchange_hash = modelutl.exchange_hash()
met_rxns = {}
for ftr in metabolite_data.features:
    met = None
    if ftr.id in model.metabolites:
        met = model.metabolites.get_by_id(ftr.id)
    else:
        mets = modelutl.find_met(ftr.id)
        if len(mets) > 0:
            for item in mets:
                if item in exchange_hash:
                    met = item
                elif item.compartment[0:1] == "c":
                    met = item
            if met == None:
                met = mets[0]
    if met != None:
        if met in exchange_hash:
            break
            original_lower = exchange_hash[met].lower_bound
            original_upper = exchange_hash[met].upper_bound
            met_rxns[ftr.id] = exchange_hash[met]
            if ftr.get_value(condition) < 0 and met_rxns[ftr.id].lower_bound == 0: 
                met_rxns[ftr.id].lower_bound = -0.001
            elif ftr.get_value(condition) > 0 and met_rxns[ftr.id].upper_bound == 0:
                met_rxns[ftr.id].upper_bound = 0.001
            print(met_rxns[ftr.id].id,original_lower,original_upper,met_rxns[ftr.id].lower_bound,met_rxns[ftr.id].upper_bound)
        else:
            break
            drains = None
            if ftr.get_value(condition) < 0:
                drains = modelutl.add_exchanges_for_metabolites([met],uptake=0.001,excretion=0)
            elif ftr.get_value(condition) > 0:
                drains = modelutl.add_exchanges_for_metabolites([met],uptake=0,excretion=0.001)
            met_rxns[ftr.id] = drains[0]
    else:
        print(ftr.id," not found!")
solution = cobra.flux_analysis.pfba(model)
print(solution.fluxes["BIOMASS_Ec_iML1515_WT_75p37M"])
#Translate data into goals
goalflux = {}
reactions = ["4HTHRK","THRS"]
for rxn in reactions:
    if solution.fluxes[rxn] == 0:
        goalflux[rxn] = 0.001
    else:
        goalflux[rxn] = solution.fluxes[rxn]
pkgmgr.getpkg("BilevelPkg").build_package(None,7)
pkgmgr.getpkg("ChangeOptPkg").build_package({
    "4HTHRK":{"flux":goalflux["4HTHRK"]*1.2,"direction":"high","objcoef":1},
    "THRS":{"flux":goalflux["THRS"]*1.2,"direction":"high","objcoef":1}
})
with open("multiomics.lp", 'w') as out:
    out.write(str(model.solver))
    out.close()
solution = model.optimize()

0.37680000000000285
binary_variable_count: 7
4HTHRK
FOUND!
THRS
FOUND!


In [7]:
data = {"id":[],"value":[]}
for var in model.solver.variables:
    data["id"].append(var.name)
    data["value"].append(var.primal)
df = pd.DataFrame(data)
df.to_csv("Solution.csv")